In [1]:
import pandas as pd
import selenium 
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from dotenv import load_dotenv
from selenium.webdriver.common.by import By
from requests_html import HTMLSession
from bs4 import BeautifulSoup
from flask import Flask, redirect, request
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import random
import string
import base64
from requests import post
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!pip install spotipy

In [3]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests-html

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [6]:
CLIENT_ID = "e185ac84569b454883fecb38e904ed33"  
CLIENT_SECRET = "d31651d9b46b46bcbab9f0cde2590277" 

In [7]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [8]:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [9]:
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

# Track ID from the URI
#'6y0igZArWVi6Iz0rj35c1Y'
track_id = '2wAJTrFhCnQyNSD3oUgTZO'

# actual GET request with proper header
r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)

In [16]:
#artist_id = '36QJpDe2go2KgaRleHCDTp'
artist_id="1eQJUHJRurFXGd3j8i8W4x"
artist_id_2 = "2YZyLoL8N0Wb9xBt1NhZWg"

# pull all artists albums
r = requests.get(BASE_URL + 'artists/' + artist_id + '/albums', 
                 headers=headers, 
                 params={'include_groups': 'album', 'limit': 50})
d = r.json()

In [10]:
r = r.json()
r

{'danceability': 0.831,
 'energy': 0.427,
 'key': 2,
 'loudness': -6.903,
 'mode': 1,
 'speechiness': 0.106,
 'acousticness': 0.0531,
 'instrumentalness': 0,
 'liveness': 0.307,
 'valence': 0.216,
 'tempo': 93.004,
 'type': 'audio_features',
 'id': '2wAJTrFhCnQyNSD3oUgTZO',
 'uri': 'spotify:track:2wAJTrFhCnQyNSD3oUgTZO',
 'track_href': 'https://api.spotify.com/v1/tracks/2wAJTrFhCnQyNSD3oUgTZO',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2wAJTrFhCnQyNSD3oUgTZO',
 'duration_ms': 235320,
 'time_signature': 4}

In [14]:
data = []   # will hold all track info
albums = [] # to keep track of duplicates

# Function to fetch albums for a given artist ID and return simplified album data
def fetch_artist_albums(artist_id):
    r = requests.get(BASE_URL + 'artists/' + artist_id + '/albums', headers=headers, params={'include_groups': 'album', 'limit': 50})
    albums_data = r.json()
    simplified_albums = []
    for album in albums_data['items']:
        album_info = {
            'name': album['name'],
            'release_date': album['release_date'],
            'total_tracks': album['total_tracks'],
            'uri': album['uri']
        }
        simplified_albums.append(album_info)
    return simplified_albums

# Fetch albums for the first artist
artist_1_albums = fetch_artist_albums(artist_id)

# Fetch albums for the second artist
artist_2_albums = fetch_artist_albums(artist_id_2)

# Print simplified album data
print("Albums for Artist 1: J.Cole")
print(artist_1_albums)

print("\nAlbums for Artist 2: Kendrick Lamar")
print(artist_2_albums)

Albums for Artist 1: J.Cole
[{'name': 'D-Day: A Gangsta Grillz Mixtape', 'release_date': '2022-03-31', 'total_tracks': 15, 'uri': 'spotify:album:3tFjhFPqGhhLrCULnPu7Wy'}, {'name': 'The Off-Season', 'release_date': '2021-05-14', 'total_tracks': 12, 'uri': 'spotify:album:4JAvwK4APPArjIsOdGoJXX'}, {'name': "Revenge Of The Dreamers III: Director's Cut", 'release_date': '2020-01-16', 'total_tracks': 30, 'uri': 'spotify:album:5PerfGh9Smd6XjwkxpmL4V'}, {'name': 'Revenge Of The Dreamers III', 'release_date': '2019-07-05', 'total_tracks': 18, 'uri': 'spotify:album:2n3quCZ0anEa46j2IveacI'}, {'name': 'KOD', 'release_date': '2018-04-20', 'total_tracks': 12, 'uri': 'spotify:album:4Wv5UAieM1LDEYVq5WmqDd'}, {'name': '4 Your Eyez Only', 'release_date': '2016-12-09', 'total_tracks': 10, 'uri': 'spotify:album:3CCnGldVQ90c26aFATC1PW'}, {'name': 'Forest Hills Drive: Live from Fayetteville, NC', 'release_date': '2016-01-28', 'total_tracks': 13, 'uri': 'spotify:album:1yZC8xZwv4gsmCZ4p4JWdI'}, {'name': '2014

In [ ]:
df = pd.DataFrame(data)   
df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,uri,track_href,analysis_url,duration_ms,time_signature,track_name,album_name,short_album_name,release_date,album_id
0,0.671,0.857,7,-5.435,1,0.2920,0.266,0.000000,0.668,0.597,...,spotify:track:1BzXvBpIFWJgu0P8P6xmP4,https://api.spotify.com/v1/tracks/1BzXvBpIFWJg...,https://api.spotify.com/v1/audio-analysis/1BzX...,309323,4,Stick (with JID & J. Cole feat. Kenny Mason & ...,D-Day: A Gangsta Grillz Mixtape,D-Day: A Gangsta Grillz Mixtape,2022-03-31,3tFjhFPqGhhLrCULnPu7Wy
1,0.842,0.403,10,-8.087,1,0.5830,0.495,0.000000,0.359,0.584,...,spotify:track:7Gn7HA4yOHKXDo6vHCiNXV,https://api.spotify.com/v1/tracks/7Gn7HA4yOHKX...,https://api.spotify.com/v1/audio-analysis/7Gn7...,173589,4,Ghetto Gods Freestyle (with EARTHGANG feat. 2 ...,D-Day: A Gangsta Grillz Mixtape,D-Day: A Gangsta Grillz Mixtape,2022-03-31,3tFjhFPqGhhLrCULnPu7Wy
2,0.832,0.700,0,-6.005,0,0.0491,0.120,0.000000,0.239,0.586,...,spotify:track:0URbdJpIV9B54W2d9yeMwt,https://api.spotify.com/v1/tracks/0URbdJpIV9B5...,https://api.spotify.com/v1/audio-analysis/0URb...,166776,4,Lifestyle (with Bas feat. A$AP Ferg),D-Day: A Gangsta Grillz Mixtape,D-Day: A Gangsta Grillz Mixtape,2022-03-31,3tFjhFPqGhhLrCULnPu7Wy
3,0.504,0.636,1,-12.926,1,0.3990,0.253,0.000000,0.211,0.713,...,spotify:track:2OSsmzINPSwQ1hLoMH70Rj,https://api.spotify.com/v1/tracks/2OSsmzINPSwQ...,https://api.spotify.com/v1/audio-analysis/2OSs...,270045,4,"Starting 5 (with Lute, Cozz & Omen)",D-Day: A Gangsta Grillz Mixtape,D-Day: A Gangsta Grillz Mixtape,2022-03-31,3tFjhFPqGhhLrCULnPu7Wy
4,0.813,0.600,9,-6.769,0,0.0602,0.115,0.000013,0.159,0.622,...,spotify:track:3p2HpO3ZgvFs97L15hGZOn,https://api.spotify.com/v1/tracks/3p2HpO3ZgvFs...,https://api.spotify.com/v1/audio-analysis/3p2H...,156944,4,Coming Down (with Ari Lennox),D-Day: A Gangsta Grillz Mixtape,D-Day: A Gangsta Grillz Mixtape,2022-03-31,3tFjhFPqGhhLrCULnPu7Wy
